In [22]:
import pandas as pd
from alpha_vantage.fundamentaldata import FundamentalData
# Private Key from Rohan's account
key = '5QIM683UCAPQI1UC'
fd = FundamentalData(key,output_format = 'pandas')

In [23]:
def statements(ticker):
    # income statement of the Company
    
    # alpha vantage function to pull income statement
    raw_is = fd.get_income_statement_annual(ticker)
    IS = raw_is[0].T[2:]
    IS.columns = list(raw_is[0].T.iloc[0])

    # balance sheet of the Company
    raw_bs = fd.get_balance_sheet_annual(ticker)
    BS = raw_bs[0].T[2:]
    BS.columns = list(raw_bs[0].T.iloc[0])

    # Cash flow statement of the Company
    raw_cf = fd.get_cash_flow_annual(ticker)
    CF = raw_cf[0].T[2:]
    CF.columns = list(raw_cf[0].T.iloc[0])
    
    return (IS,BS,CF)

In [24]:
# Defining functions to calculate individual Piotroski Score Metrics

def net_income():
    df=IS
    return float(df.loc['netIncome'][0])

def roa():
    df=BS
    p,q=float(df.loc['totalAssets'][0]),float(df.loc['totalAssets'][1])
    av_assets=(p+q)/2
    return net_income()/av_assets

def ocf():
    df=CF
    return float(df.loc['operatingCashflow'][0])

def ltdebt():
    df=BS
    p,q=float(df.loc['longTermDebt'][0]),float(df.loc['longTermDebt'][1])
    return q-p

def current_ratio():
    df=BS
    p,q=float(df.loc['totalCurrentAssets'][0]),float(df.loc['totalCurrentAssets'][1])
    r,s=float(df.loc['totalCurrentLiabilities'][0]),float(df.loc['totalCurrentLiabilities'][1])
    current_ratio1,current_ratio2=p/r,q/s
    return current_ratio1-current_ratio2

def new_shares():
    return float(BS.loc['commonStock'][0]) - float(BS.loc['commonStock'][1])

def gross_margin():
    df=IS
    gross_margin_this = float(IS.loc['grossProfit'][0])/float(IS.loc['totalRevenue'][0])
    gross_margin_last = float(IS.loc['grossProfit'][1])/float(IS.loc['totalRevenue'][1])
    return gross_margin_this - gross_margin_last

def Asset_Turnover_Ratio():
    df1=IS
    df2=BS
    p,q,s=float(df2.loc['totalAssets'][0]),float(df2.loc['totalAssets'][1]),float(df2.loc['totalAssets'][2])
    av_assets1=(p+q)/2
    av_assets2=(q+s)/2
    atr1=float(df1.loc['totalRevenue'][0])/av_assets1
    atr2=float(df1.loc['totalRevenue'][1])/av_assets2
    return atr1-atr2

In [25]:
# Piotroski Function definition
def Piotroski_Score():
    Pscore=0
    if net_income()>0:
        Pscore+=1
    if roa()>0:
        Pscore+=1
    if ocf()>0:
        Pscore+=1
    if ocf()>net_income():
        Pscore+=1
    if ltdebt()>0:
        Pscore+=1
    if current_ratio()>0:
        Pscore+=1
    if new_shares()>0:
        Pscore+=1
    if gross_margin()>0:
        Pscore+=1
    if Asset_Turnover_Ratio()>0:
        Pscore+=1
    return Pscore

In [29]:
# Input the list of stocks you would want to see here. 
# A Free account has API call limitations. Use a premium account to get the Piotroski Scores of multiple stocks
tickers=['AAPL']
data2={}
for ticker in tickers:
    IS,BS,CF = statements(ticker)
    data2[ticker]=Piotroski_Score()
    data=pd.DataFrame(data2,index=[0])    


In [30]:
data

,AAPL
0,7
